In [1]:
import cv2
import math
import numpy as np
from  ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 2050


In [3]:
model_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\Yolo models\yolov8m.pt"
video_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\test_data\test_video\cars.mp4"

In [4]:
model = YOLO(model_path)

In [5]:
tracker = DeepSort(max_age=30)

c:\Users\Rishabh Surana\Desktop\ATMS project\venv-gpu\Lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [6]:
cap = cv2.VideoCapture(video_path)

In [7]:
if not cap.isOpened():
    print("Error : Could not open video")
    exit()
else:
    print("Video is opened and working")

Video is opened and working


In [8]:
meters_per_pixel = 5/50
print(meters_per_pixel)

0.1


In [9]:
def obj_Detection(results, detections):
    for box in results.boxes:
        x1, y1, x2, y2 = map(int,box.xyxy[0])
        class_id = int(box.cls[0])
        class_name = results.names[class_id]
        confidence = float(box.conf[0])
        
        detections.append(([x1,y1,x2-x1,y2-y1], confidence, class_name))

In [10]:
stall_cars = []
def stall_movement_fn(car_coordinates, car_id):
    def eucledian(base_coord, pt):
        pt1 = (base_coord[0] - pt[0])**2
        pt2 = (base_coord[1] - pt[1])**2
        dist = math.sqrt(pt1+pt2)
        return dist

    coords = car_coordinates[car_id]
    stalled = True
    if(len(coords) > 10):
        recent_coords = coords[-10:]
        base_coord = recent_coords[0]
        for pt in recent_coords[1:]:
            distance = eucledian(base_coord, pt)
            if distance >= 5: #pixel_threshold
                stalled = False
                break
    else:
        stalled = False
        
    return stalled
    
    

In [11]:
def alert(car_id, reason):
    if(reason=="Opposite_direction"):
        print(f"{car_id} is moving in Opposite Direction.")
    elif(reason=="Stalled"):
        print(f"{car_id} is stalled.")
    elif(reason=="Overspeed"):
        print(f"{car_id} is Overspeeding.")
    else:
        print(f"{car_id} is ok")

In [12]:
opposite_cars = []
def detect_car_movement(car_coordinates, expected = [0,-1], threshold = 0.4):
    
    def normalize_vector(v):
        mag = math.sqrt(v[0]**2 + v[1]**2)
        if(mag!=0):
            return [v[0]/mag, v[1]/mag]
        else:
            return [0,0]
        
    def cosine_similarity(movment_vec, expected):
        dot_num = (movment_vec[0] * expected[0]) + (movment_vec[1] * expected[1])
        mag1 = math.sqrt((movment_vec[0]**2)+(movment_vec[1]**2))
        mag2 = math.sqrt((expected[0]**2)+(expected[1]**2))
        if(mag1 and mag2):
            return dot_num / (mag1 * mag2)
        else:
            return 1
    
    for car_id, coords in car_coordinates.items():
        
        if(len(coords)<2):
            continue
        
        x1, y1 = coords[-2] 
        x2, y2 = coords[-1]
        
        if(x2-x1 == 0 and y2-y1 == 0):
            if(stall_movement_fn(car_coordinates, car_id)):
                if(car_id not in stall_cars):
                    stall_cars.append(car_id)
                    alert(car_id,"Stalled")
        movment = [x2-x1, y2-y1]
        movement_unit = normalize_vector(movment)
        similarity = cosine_similarity(movement_unit, expected)
        
        
        if(similarity < -threshold):
            if(car_id not in opposite_cars):
                alert(car_id,"Opposite_direction")
                opposite_cars.append(car_id)
        

In [13]:
def calculate_speed(car_coordinates, fps, meters_per_pixel):
    speeds = []
    for i in range(1, len(car_coordinates)):
        x1, y1 = car_coordinates[i-1]
        x2, y2 = car_coordinates[i]
        pixel_dist = math.hypot(x2 - x1, y2 - y1)
        distance_m = pixel_dist * meters_per_pixel
        time_s = 1 / fps
        speed_mps = distance_m / time_s
        speed_kmph = speed_mps * 3.6
        speeds.append(speed_kmph)
    if speeds:
        return sum(speeds)/len(speeds)
    else:
        return 0.0

In [14]:
car_coordinates = {}
def car_coords_fn(track_id, x1, y1, x2, y2):
    center_x = int((x1 + x2)/2)    
    center_y = int((y1 + y2)/2)
    if track_id not in car_coordinates:
        car_coordinates[track_id] = []
    if len(car_coordinates[track_id]) >= 20:
        car_coordinates[track_id].pop(0)
    car_coordinates[track_id].append([center_x,center_y])
    detect_car_movement(car_coordinates)

In [15]:
overspeed = {}
def draw_Tracks(tracks,frame):
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        class_name = track.get_det_class()
        
        car_coords_fn(track_id, x1, y1, x2, y2)
        speed = calculate_speed(car_coordinates[track_id], fps = 30, meters_per_pixel=meters_per_pixel)
        if(speed > 60 and track_id not in overspeed):
            alert(track_id,"Overspeed")
            overspeed[track_id] = speed
            
        label = f"ID {track_id} | {class_name} | {speed:.1f} km/h"
        
        if track_id in stall_cars:
            color = (0,255,255)
        elif track_id in opposite_cars:
            color = (0,0,255)
        elif speed > 60:
            color = (0,165,255)
        else:
            color = (0,255,0)
        
        
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 1)
        cv2.putText(frame, label, (x1, y1 - 10,), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        

In [16]:
while True:
   ret, frame = cap.read()
   if not ret:
       break
#    frame = bev_func(frame)
   results = model(frame, conf = 0.6)[0]
   detections = []
   
   obj_Detection(results, detections)
   
   tracks = tracker.update_tracks(detections, frame = frame)
   
   draw_Tracks(tracks,frame)
   
   cv2.imshow("Output", frame)
   if(cv2.waitKey(1) and 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()
   
   


0: 384x640 1 person, 5 cars, 1 truck, 68.6ms
Speed: 20.2ms preprocess, 68.6ms inference, 35.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 truck, 29.9ms
Speed: 1.8ms preprocess, 29.9ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 truck, 35.7ms
Speed: 1.8ms preprocess, 35.7ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 truck, 36.1ms
Speed: 2.1ms preprocess, 36.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
1 is Overspeeding.
2 is moving in Opposite Direction.
4 is Overspeeding.

0: 384x640 1 person, 5 cars, 1 truck, 35.5ms
Speed: 1.8ms preprocess, 35.5ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 truck, 44.2ms
Speed: 1.8ms preprocess, 44.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 truck, 42.8ms
Speed: 1.3ms preproce

In [17]:
total_cars = len(car_coordinates)
print(f"Total Cars = {total_cars}")
print("--"*130)

for track_id, coords in car_coordinates.items():
    print(f"\nCar Id {track_id} -> {coords}")
    speed = calculate_speed(coords, fps = 30, meters_per_pixel=meters_per_pixel)
    print(f"Speed of car Id {track_id} : {speed:.2f} km/h.")

Total Cars = 80
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Car Id 1 -> [[303, 23], [303, 22], [302, 21], [302, 21], [302, 21], [301, 20], [301, 20], [301, 19], [301, 19], [301, 19], [301, 18], [301, 18], [300, 17], [300, 16], [300, 16], [299, 16], [299, 15], [299, 15], [299, 15], [298, 14]]
Speed of car Id 1 : 6.63 km/h.

Car Id 2 -> [[-17, 215], [-19, 216], [-21, 218], [-23, 220], [-25, 222], [-27, 224], [-29, 226], [-32, 228], [-34, 230], [-36, 232], [-38, 234], [-40, 236], [-42, 238], [-44, 240], [-46, 242], [-48, 244], [-51, 246], [-53, 248], [-55, 249], [-57, 251]]
Speed of car Id 2 : 30.76 km/h.

Car Id 3 -> [[251, 13], [251, 13], [251, 13], [251, 12], [251, 12], [251, 12], [251, 12], [251, 12], [251, 11], [252, 11], [252, 11], [252, 11], [252, 11], [252, 10], [252

In [18]:
print("\n---Stalled Vehicle---\n")
for s_car in stall_cars:
    print(f"Car Id {s_car} Stalled.")
print("\n\n---Opposite Vehicle---\n")
for op_car in opposite_cars:
    print(f"Car Id {op_car} is moving in opposite direction.")
print("\n\n---Overspeed Vehicle---\n")
for overspd in overspeed:
    print(f"Car Id {overspd} is overspeeding.")


---Stalled Vehicle---

Car Id 3 Stalled.
Car Id 6 Stalled.
Car Id 7 Stalled.
Car Id 5 Stalled.
Car Id 1 Stalled.
Car Id 9 Stalled.
Car Id 17 Stalled.
Car Id 11 Stalled.
Car Id 14 Stalled.
Car Id 24 Stalled.
Car Id 18 Stalled.
Car Id 20 Stalled.
Car Id 22 Stalled.
Car Id 26 Stalled.
Car Id 38 Stalled.
Car Id 29 Stalled.
Car Id 32 Stalled.
Car Id 34 Stalled.
Car Id 36 Stalled.
Car Id 50 Stalled.
Car Id 40 Stalled.
Car Id 43 Stalled.
Car Id 47 Stalled.
Car Id 49 Stalled.
Car Id 55 Stalled.
Car Id 51 Stalled.
Car Id 53 Stalled.
Car Id 54 Stalled.
Car Id 67 Stalled.
Car Id 59 Stalled.
Car Id 61 Stalled.
Car Id 64 Stalled.
Car Id 69 Stalled.
Car Id 71 Stalled.
Car Id 73 Stalled.
Car Id 88 Stalled.
Car Id 89 Stalled.
Car Id 74 Stalled.
Car Id 82 Stalled.
Car Id 81 Stalled.
Car Id 86 Stalled.
Car Id 94 Stalled.
Car Id 109 Stalled.
Car Id 98 Stalled.
Car Id 119 Stalled.
Car Id 100 Stalled.
Car Id 121 Stalled.
Car Id 122 Stalled.
Car Id 125 Stalled.
Car Id 107 Stalled.
Car Id 112 Stalled.
Car I